# Installing and importing dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
!pip install tensorflow==1.14.0 
!pip install keras==2.1.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 109.3 MB 42 kB/s 
     |████████████████████████████████| 488 kB 45.0 MB/s 
     |████████████████████████████████| 50 kB 5.6 MB/s 
     |████████████████████████████████| 3.1 MB 41.7 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.2+zzzcolab20220719082949
    Uninstalling tensorflow-2.8.2+zzzcolab20220719082949:
      Successfully uninstalled tensorflow-2.8.2+zzzcolab20220719082949
ERROR: pip's dependency resolver does not currently ta

In [3]:
import pandas as pd
import collections
import gensim
from gensim.models.keyedvectors import KeyedVectors
from gensim.models.fasttext import FastText
from gensim.models import word2vec

from sklearn.model_selection import train_test_split,cross_val_score, cross_val_predict, KFold, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix, precision_recall_fscore_support

#import pandas as pd
import numpy as np

#Add My
#from pandas import Series
#from numpy.random import randn

import keras
from keras import backend as K
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.contrib.layers.python.layers import initializers

Using TensorFlow backend.
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [4]:
#set folder paths
#folder_path = ''
#context = 5

#data_path = folder_path + '/content/drive/MyDrive/Sinhala review dataset form/DataSet/FinalDataSet/CommentsLB3columnDocid  .csv'

In [5]:
#set folder paths
folder_path = ''
EMBEDDING_SIZE = 300 
embedding_type = "fasttext"
context = 5

edp = folder_path + '/content/drive/MyDrive/Sinhala review dataset form/DataSet/FinalDataSet/CommentsLB3columnDocid  .csv'
gdp = folder_path + '/content/drive/MyDrive/Sinhala review dataset form/DataSet/FinalDataSet/CommentsLB3columnDocid  .csv'

word_embedding_path = folder_path + "word_embedding/"+embedding_type+"/source2_data_from_ec_and_gf/"+str(EMBEDDING_SIZE)+"/"+embedding_type+"_"+str(EMBEDDING_SIZE)+"_"+str(context)
word_embedding_keydvectors_path = folder_path + "word_embedding/"+embedding_type+"/source2_data_from_ec_and_gf/"+str(EMBEDDING_SIZE)+"/keyed_vectors/keyed.kv"
embedding_matrix_path = folder_path + 'Sentiment Analysis/CNN RNN/embedding_matrix/'+embedding_type+'ec_gf'+str(EMBEDDING_SIZE)+'_'+str(context)

In [6]:
#word_embedding_path

'word_embedding/fasttext/source2_data_from_ec_and_gf/300/fasttext_300_5'

# Load Dataset


In [7]:
#dataPath = pd.read_csv(data_path)
#dataPath.shape

In [8]:
#ecd = pd.read_csv(edp, encoding= 'unicode_escape')
#gfd = pd.read_csv(gdp, encoding= 'unicode_escape')

In [9]:
ecd = pd.read_csv(edp)
gfd = pd.read_csv(gdp)

In [10]:
ecd
#dataPath

,docid,Comments,Lable
0,100204,භාන්ඩය ඉතමත් හොද තත්වයක පවතුනි. නියමිත ප්‍රමිත...,5
1,100205,භාන්ඩය ඉතා හොද තත්වයෙ පවතී.,5
2,100206,මෙම වෙබ් අඩවියේ පාරිබෝගික සේවාව ඉතා ඉහළ මට්ටමක...,5
3,100207,දුරකථන කවරය ඉතා හොද ප්‍රමිතියකට නිමවා ඇත.,5
4,100208,මා බලා පොරොත්තු වූ වෙලද නාමය යටතේම ඇති අයිතම ඉ...,5
...,...,...,...
1225,100445,විස්තරයේ පෙන්වා ඇති පරිදි වේගවත් ආරෝපණයක් ගෙන ...,1
1226,100446,ඒක කකුලටවත් ඇඟටවත් ගැළපෙන්නේ නැහැ.,1
1227,100447,"භාණ්ඩ කිසි විටෙකත් යවා නැත, මට භාන්ඩය එවීම පිල...",1
1228,100448,සායක් ඉතා කෙටි හා ඉතා බාල රෙදි වර්ගයකින් නිමවා...,1


In [11]:
ecd.isnull().sum()

docid       0
Comments    2
Lable       0
dtype: int64

In [12]:
ecd['Comments'].isnull().sum()

2

In [13]:
modifiedEcdDataPath = ecd.dropna()

In [14]:
modifiedEcdDataPath.isnull().sum()

docid       0
Comments    0
Lable       0
dtype: int64

In [15]:
modifiedEcdDataPath.to_csv('modifiedDataPath.csv',index=False)

In [16]:
gfd.shape

(1230, 3)

In [17]:
gfd.isnull().sum()

docid       0
Comments    2
Lable       0
dtype: int64

In [18]:
gfd['Comments'].isnull().sum()

2

In [19]:
modifiedGfdDataPath = gfd.dropna()

In [20]:
modifiedGfdDataPath.isnull().sum()

docid       0
Comments    0
Lable       0
dtype: int64

In [21]:
modifiedGfdDataPath.to_csv('modifiedDataPath.csv',index=False)

In [22]:
modifiedGfdDataPath

,docid,Comments,Lable
0,100204,භාන්ඩය ඉතමත් හොද තත්වයක පවතුනි. නියමිත ප්‍රමිත...,5
1,100205,භාන්ඩය ඉතා හොද තත්වයෙ පවතී.,5
2,100206,මෙම වෙබ් අඩවියේ පාරිබෝගික සේවාව ඉතා ඉහළ මට්ටමක...,5
3,100207,දුරකථන කවරය ඉතා හොද ප්‍රමිතියකට නිමවා ඇත.,5
4,100208,මා බලා පොරොත්තු වූ වෙලද නාමය යටතේම ඇති අයිතම ඉ...,5
...,...,...,...
1225,100445,විස්තරයේ පෙන්වා ඇති පරිදි වේගවත් ආරෝපණයක් ගෙන ...,1
1226,100446,ඒක කකුලටවත් ඇඟටවත් ගැළපෙන්නේ නැහැ.,1
1227,100447,"භාණ්ඩ කිසි විටෙකත් යවා නැත, මට භාන්ඩය එවීම පිල...",1
1228,100448,සායක් ඉතා කෙටි හා ඉතා බාල රෙදි වර්ගයකින් නිමවා...,1


In [23]:
all_data = pd.concat([modifiedEcdDataPath,modifiedGfdDataPath], ignore_index=True)
all_data.shape

(2456, 3)

# Process data

In [24]:
def text_preprocessing_2(data):
  comments = data['Comments']
  labels = data['Lable']
  comments_splitted = []

  for comment in comments:
    lines = []
    try:
      words = comment.split()
      lines += words
    except:
      continue
    comments_splitted.append(lines)

  return comments_splitted,labels

In [25]:
comment_texts, comment_labels = text_preprocessing_2(all_data)

# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(comment_texts)
vocab_size = len(t.word_index) + 1
print(vocab_size)
#comment_labels
#comment_texts

3316


In [26]:
encoded_docs = t.texts_to_sequences(comment_texts)
#encoded_docs

max_length = len(max(encoded_docs, key=len))
padded_docs = pad_sequences(encoded_docs, maxlen=max_length,padding='post')
comment_labels = np.array(comment_labels)
padded_docs = np.array(padded_docs)
comment_labels
#padded_docs
#max_length

In [27]:
max_length = len(max(encoded_docs, key=len))
padded_docs = pad_sequences(encoded_docs, maxlen=max_length,padding='post')
comment_labels = np.array(comment_labels)
padded_docs = np.array(padded_docs)
#padded_docs
#comment_labels

In [28]:
comment_labels = pd.get_dummies(comment_labels).values
#comment_labels

In [29]:
X_train, X_test, y_train, y_test = train_test_split(padded_docs, comment_labels, test_size=0.1, random_state=0)

Load Embeddings
# New section

In [30]:
#set embedding path to load embeddings
embedding_path = ''

In [31]:
def generate_embedding_metrix():
  if (embedding_type == 'fasttext'):
    word_embedding_model = FastText.load(embedding_path)
  else:
    word_embedding_model = word2vec.Word2Vec.load(word_embedding_path)
    
  word_vectors = word_embedding_model.wv
  word_vectors.save(word_embedding_keydvectors_path)
  word_vectors = KeyedVectors.load(word_embedding_keydvectors_path, mmap='r')

  embeddings_index = dict()
  for word, vocab_obj in word_vectors.vocab.items():
    embeddings_index[word]=word_vectors[word]

  # create a weight matrix for words in training docs
  embedding_matrix = zeros((vocab_size, embedding_size))
  for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
      embedding_matrix[i] = embedding_vector

  pickle.dump(embedding_matrix, open(embedding_matrix_path, 'wb'))
  return embedding_matrix

In [32]:
# embedding_vectors = generate_embedding_metrix()

FileNotFoundError: ignored

# Capsule Network Model

In [33]:
def _conv2d_wrapper(inputs, shape, strides, padding, add_bias, activation_fn, name, stddev=0.1):
  with tf.variable_scope(name,reuse=tf.AUTO_REUSE) as scope:
    kernel = _get_weights_wrapper(
      name='weights', shape=shape, weights_decay_factor=0.0, )
    output = tf.nn.conv2d(inputs, filter=kernel, strides=strides, padding=padding, name='conv')
    if add_bias:
      biases = _get_biases_wrapper(name='biases', shape=[shape[-1]] )
      output = tf.add(output, biases, name='biasAdd')
    if activation_fn is not None:
      output = activation_fn(output, name='activation')
  return output

In [34]:
def _get_weights_wrapper(name, shape, dtype=tf.float32, initializer=initializers.xavier_initializer(),weights_decay_factor=None):
  weights = _get_variable_wrapper(name=name, shape=shape, dtype=dtype, initializer=initializer)
  if weights_decay_factor is not None and weights_decay_factor > 0.0:
    weights_wd = tf.multiply(tf.nn.l2_loss(weights), weights_decay_factor, name=name + '/l2loss')
    tf.add_to_collection('losses', weights_wd)
  return weights

In [35]:
def _get_biases_wrapper(name, shape, dtype=tf.float32, initializer=tf.constant_initializer(0.0)):
  """Wrapper over _get_variable_wrapper() to get bias.
  """
  biases = _get_variable_wrapper(name=name, shape=shape, dtype=dtype, initializer=initializer)
  return biases

In [36]:
def _get_variable_wrapper(name, shape=None, dtype=None, initializer=None,regularizer=None,trainable=True,collections=None,caching_device=None,partitioner=None,validate_shape=True,custom_getter=None):
  with tf.device('/cpu:0'):
    var = tf.get_variable(
      name, shape=shape, dtype=dtype, initializer=initializer,
      regularizer=regularizer, trainable=trainable,
      collections=collections, caching_device=caching_device,
      partitioner=partitioner, validate_shape=validate_shape,
      custom_getter=custom_getter
    )
  return var

In [37]:
def softmax(x, axis=-1):
    ex = K.exp(x - K.max(x, axis=axis, keepdims=True))
    return ex/K.sum(ex, axis=axis, keepdims=True)

In [38]:
def squash_v1(x, axis=-1):
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True) + K.epsilon()
    scale = K.sqrt(s_squared_norm)/ (0.5 + s_squared_norm)
    return scale * x

In [39]:
def squash_v0(s, axis=-1, epsilon=1e-7, name=None):
    s_squared_norm = K.sum(K.square(s), axis, keepdims=True) + K.epsilon()
    safe_norm = K.sqrt(s_squared_norm)
    scale = 1 - tf.exp(-safe_norm)
    return scale * s / safe_norm

In [40]:
def routing(u_hat_vecs, beta_a, iterations, output_capsule_num, i_activations):
    b = keras.backend.zeros_like(u_hat_vecs[:,:,:,0])
    if i_activations is not None:
        i_activations = i_activations[...,tf.newaxis]
    for i in range(iterations):
        if False:
            leak = tf.zeros_like(b, optimize=True)
            leak = tf.reduce_sum(leak, axis=1, keep_dims=True)
            leaky_logits = tf.concat([leak, b], axis=1)
            leaky_routing = tf.nn.softmax(leaky_logits, dim=1)        
            c = tf.split(leaky_routing, [1, output_capsule_num], axis=1)[1]
        else:
            c = softmax(b, 1)   
        outputs = squash_v1(K.batch_dot(c, u_hat_vecs, [2, 2]))
        if i < iterations - 1:
            b = b + K.batch_dot(outputs, u_hat_vecs, [2, 3])                                    
    poses = outputs 
    activations = K.sqrt(K.sum(K.square(poses), 2))
    return poses, activations

In [41]:
def vec_transformationByConv(poses, input_capsule_dim, input_capsule_num, output_capsule_dim, output_capsule_num):                            
    kernel = _get_weights_wrapper(name='weights', shape=[1, input_capsule_dim, output_capsule_dim*output_capsule_num], weights_decay_factor=0.0)
    u_hat_vecs = keras.backend.conv1d(poses, kernel)
    u_hat_vecs = keras.backend.reshape(u_hat_vecs, (-1, input_capsule_num, output_capsule_num, output_capsule_dim))
    u_hat_vecs = keras.backend.permute_dimensions(u_hat_vecs, (0, 2, 1, 3))
    return u_hat_vecs

In [42]:
def vec_transformationByMat(poses, input_capsule_dim, input_capsule_num, output_capsule_dim, output_capsule_num, shared=True):                        
    inputs_poses_shape = poses.get_shape().as_list()
    poses = poses[..., tf.newaxis, :]        
    poses = tf.tile(poses, [1, 1, output_capsule_num, 1])    
    if shared:
        kernel = _get_weights_wrapper(name='weights', shape=[1, 1, output_capsule_num, output_capsule_dim, input_capsule_dim], weights_decay_factor=0.0)
        kernel = tf.tile(kernel, [inputs_poses_shape[0], input_capsule_num, 1, 1, 1])
    else:
        kernel = _get_weights_wrapper(name='weights', shape=[1, input_capsule_num, output_capsule_num, output_capsule_dim, input_capsule_dim], weights_decay_factor=0.0)
        kernel = tf.tile(kernel, [inputs_poses_shape[0], 1, 1, 1, 1])
    u_hat_vecs = tf.squeeze(tf.matmul(kernel, poses[...,tf.newaxis]),axis=-1)
    u_hat_vecs = keras.backend.permute_dimensions(u_hat_vecs, (0, 2, 1, 3))
    return u_hat_vecs

In [43]:
def capsules_init(inputs, shape, strides, padding, pose_shape, add_bias, name):
    with tf.variable_scope(name,reuse=tf.AUTO_REUSE):   
        poses = _conv2d_wrapper(
          inputs,
          shape=shape[0:-1] + [shape[-1] * pose_shape],
          strides=strides,
          padding=padding,
          add_bias=add_bias,
          activation_fn=None,
          name='pose_stacked'
        )        
        poses_shape = poses.get_shape().as_list()    
        poses = tf.reshape(poses, [-1, poses_shape[1], poses_shape[2], shape[-1], pose_shape])        
        beta_a = _get_weights_wrapper(name='beta_a', shape=[1, shape[-1]])    
        poses = squash_v1(poses, axis=-1)  
        activations = K.sqrt(K.sum(K.square(poses), axis=-1)) + beta_a        

    return poses, activations

In [44]:
def capsule_fc_layer(nets, output_capsule_num, iterations, name):
    with tf.variable_scope(name,reuse=tf.AUTO_REUSE):   
        poses, i_activations = nets
        input_pose_shape = poses.get_shape().as_list()

        u_hat_vecs = vec_transformationByConv(poses,input_pose_shape[-1], input_pose_shape[1],input_pose_shape[-1], output_capsule_num,)
        beta_a = _get_weights_wrapper(name='beta_a', shape=[1, output_capsule_num])
        poses, activations = routing(u_hat_vecs, beta_a, iterations, output_capsule_num, i_activations)
    return poses, activations

In [45]:
def capsule_flatten(nets):
    poses, activations = nets
    input_pose_shape = poses.get_shape().as_list()
    
    poses = tf.reshape(poses, [
                    -1, input_pose_shape[1]*input_pose_shape[2]*input_pose_shape[3], input_pose_shape[-1]]) 
    activations = tf.reshape(activations, [
                    -1, input_pose_shape[1]*input_pose_shape[2]*input_pose_shape[3]])
    return poses, activations

In [46]:
def capsule_conv_layer(nets, shape, strides, iterations, name):   
    with tf.variable_scope(name,reuse=tf.AUTO_REUSE):              
        poses, i_activations = nets
        
        inputs_poses_shape = poses.get_shape().as_list()

        hk_offsets = [
          [(h_offset + k_offset) for k_offset in range(0, shape[0])] for h_offset in
          range(0, inputs_poses_shape[1] + 1 - shape[0], strides[1])
        ]
        wk_offsets = [
          [(w_offset + k_offset) for k_offset in range(0, shape[1])] for w_offset in
          range(0, inputs_poses_shape[2] + 1 - shape[1], strides[2])
        ]
    
        inputs_poses_patches = tf.transpose(
          tf.gather(
            tf.gather(
              poses, hk_offsets, axis=1, name='gather_poses_height_kernel'
            ), wk_offsets, axis=3, name='gather_poses_width_kernel'
          ), perm=[0, 1, 3, 2, 4, 5, 6], name='inputs_poses_patches'
        )
        inputs_poses_shape = inputs_poses_patches.get_shape().as_list()
        inputs_poses_patches = tf.reshape(inputs_poses_patches, [
                                -1, shape[0]*shape[1]*shape[2], inputs_poses_shape[-1]
                                ])

        i_activations_patches = tf.transpose(
          tf.gather(
            tf.gather(
              i_activations, hk_offsets, axis=1, name='gather_activations_height_kernel'
            ), wk_offsets, axis=3, name='gather_activations_width_kernel'
          ), perm=[0, 1, 3, 2, 4, 5], name='inputs_activations_patches'
        )
        i_activations_patches = tf.reshape(i_activations_patches, [
                                -1, shape[0]*shape[1]*shape[2]]
                                )
        u_hat_vecs = vec_transformationByConv(
                  inputs_poses_patches,
                  inputs_poses_shape[-1], shape[0]*shape[1]*shape[2],
                  inputs_poses_shape[-1], shape[3],
                  )  
        beta_a = _get_weights_wrapper(
                name='beta_a', shape=[1, shape[3]]
                )
        poses, activations = routing(u_hat_vecs, beta_a, iterations, shape[3], i_activations_patches)
        poses = tf.reshape(poses, [
                    inputs_poses_shape[0], inputs_poses_shape[1],
                    inputs_poses_shape[2], shape[3],
                    inputs_poses_shape[-1]]
                ) 
        activations = tf.reshape(activations, [
                    inputs_poses_shape[0],inputs_poses_shape[1],
                    inputs_poses_shape[2],shape[3]]
                ) 
        nets = poses, activations            
    return nets

In [47]:
def capsule_model_A(X, num_classes):
    with tf.variable_scope('capsule_'+str(3),reuse=tf.AUTO_REUSE ):   
        nets = _conv2d_wrapper(
                X, shape=[3, 300, 1, 32], strides=[1, 2, 1, 1], padding='VALID', 
                add_bias=True, activation_fn=tf.nn.relu, name='conv1'
            )
        nets = capsules_init(nets, shape=[1, 1, 32, 16], strides=[1, 1, 1, 1], 
                             padding='VALID', pose_shape=16, add_bias=True, name='primary')                        
        nets = capsule_conv_layer(nets, shape=[3, 1, 16, 16], strides=[1, 1, 1, 1], iterations=3, name='conv2')
        nets = capsule_flatten(nets)
        poses, activations = capsule_fc_layer(nets, num_classes, 3, 'fc2') 
    return poses, activations

In [48]:
def capsule_model_A(X, num_classes):
    with tf.variable_scope('capsule_'+str(3),reuse=tf.AUTO_REUSE ):   
        nets = _conv2d_wrapper(
                X, shape=[3, 300, 1, 32], strides=[1, 2, 1, 1], padding='VALID', 
                add_bias=True, activation_fn=tf.nn.relu, name='conv1'
            )
        nets = capsules_init(nets, shape=[1, 1, 32, 16], strides=[1, 1, 1, 1], 
                             padding='VALID', pose_shape=16, add_bias=True, name='primary')                        
        nets = capsule_conv_layer(nets, shape=[3, 1, 16, 16], strides=[1, 1, 1, 1], iterations=3, name='conv2')
        nets = capsule_flatten(nets)
        poses, activations = capsule_fc_layer(nets, num_classes, 3, 'fc2') 
    return poses, activations

In [49]:
def capsule_model_B(X, num_classes):
    poses_list = []
    for _, ngram in enumerate([3,4,5]):
        with tf.variable_scope('capsule_'+str(ngram),reuse=tf.AUTO_REUSE): 
            nets = _conv2d_wrapper(
                X, shape=[ngram, 300, 1, 32], strides=[1, 2, 1, 1], padding='VALID', 
                add_bias=True, activation_fn=tf.nn.relu, name='conv1'
            )
            nets = capsules_init(nets, shape=[1, 1, 32, 16], strides=[1, 1, 1, 1], 
                                 padding='VALID', pose_shape=16, add_bias=True, name='primary')                        
            nets = capsule_conv_layer(nets, shape=[3, 1, 16, 16], strides=[1, 1, 1, 1], iterations=3, name='conv2')
            nets = capsule_flatten(nets)
            poses, activations = capsule_fc_layer(nets, num_classes, 3, 'fc2')
            poses_list.append(poses)
    
    poses = tf.reduce_mean(tf.convert_to_tensor(poses_list), axis=0) 
    activations = K.sqrt(K.sum(K.square(poses), 2))
    return poses, activations

Loss functions

In [50]:
def spread_loss(labels, activations, margin):
    activations_shape = activations.get_shape().as_list()
    mask_t = tf.equal(labels, 1)
    mask_i = tf.equal(labels, 0)    
    activations_t = tf.reshape(
      tf.boolean_mask(activations, mask_t), [activations_shape[0], 1]
    )    
    activations_i = tf.reshape(
      tf.boolean_mask(activations, mask_i), [activations_shape[0], activations_shape[1] - 1]
    )    
    gap_mit = tf.reduce_sum(tf.square(tf.nn.relu(margin - (activations_t - activations_i))))
    return gap_mit   

In [51]:
def cross_entropy(y, preds):    
    y = tf.argmax(y, axis=1)
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=preds, labels=y)                                               
    loss = tf.reduce_mean(loss) 
    return loss

In [52]:
def margin_loss(y, preds):    
    y = tf.cast(y,tf.float32)
    loss = y * tf.square(tf.maximum(0., 0.9 - preds)) + \
        0.25 * (1.0 - y) * tf.square(tf.maximum(0., preds - 0.1))
    loss = tf.reduce_mean(tf.reduce_sum(loss, axis=1))
    return loss

Training process

In [53]:
class Args:
  embedding_type = "static"
  dataset = "/content/modifiedDataPath.csv"
  loss_type = "margin_loss"
  model_type = "capsule-B"
  has_test = 1
  has_dev = 1
  num_epochs = 10
  batch_size = 64
  use_orphan = False
  use_leaky = False
  learning_rate = 0.001
  margin = 0.2
  num_classes = 5
  vocab_size = vocab_size
  vec_size = 300
  max_sent = max_length

In [54]:
args = Args()

In [55]:
with tf.device('/cpu:0'):
    global_step = tf.train.get_or_create_global_step()

In [56]:
class BatchGenerator(object):
    """Generate and hold batches."""
    def __init__(self, dataset,label, batch_size,input_size, is_shuffle=True):
      self._dataset = dataset
      self._label = label
      self._batch_size = batch_size    
      self._cursor = 0      
      self._input_size = input_size      
      
      if is_shuffle:
          index = np.arange(len(self._dataset))
          np.random.shuffle(index)
          self._dataset = np.array(self._dataset)[index]
          self._label = np.array(self._label)[index]
      else:
          self._dataset = np.array(self._dataset)
          self._label = np.array(self._label)
    def next(self):
      if self._cursor + self._batch_size > len(self._dataset):
          self._cursor = 0
      """Generate a single batch from the current cursor position in the data."""      
      batch_x = self._dataset[self._cursor : self._cursor + self._batch_size,:]
      batch_y = self._label[self._cursor : self._cursor + self._batch_size]
      self._cursor += self._batch_size
      return batch_x, batch_y

In [57]:
best_model = None
best_epoch = 0
best_acc_val = 0.

In [58]:
lr = args.learning_rate
m = args.margin

In [60]:
acc_per_fold = []
precision_per_fold = []
recall_per_fold = []
f1_per_fold = []

kfold = KFold(n_splits=10, shuffle=True)

fold_no = 1
inputs = padded_docs
targets = comment_labels

for train, test in kfold.split(inputs, targets):
  

    n_iterations_per_epoch = len(inputs[train]) // args.batch_size
    n_iterations_test = len(inputs[test]) // args.batch_size

    mr_train1 = BatchGenerator(inputs[train], targets[train], args.batch_size, 0)    
    mr_test1 = BatchGenerator(inputs[test], targets[test], args.batch_size, 0, is_shuffle=False)
    best_accuracy = 0.
    best_precision = 0.
    best_recall = 0.
    best_f1 = 0.

    X = tf.placeholder(tf.int32, [args.batch_size, args.max_sent], name="input_x")
    y = tf.placeholder(tf.int64, [args.batch_size, args.num_classes], name="input_y")
    is_training = tf.placeholder_with_default(False, shape=())    
    learning_rate = tf.placeholder(dtype='float32') 
    margin = tf.placeholder(shape=(),dtype='float32') 

    l2_loss = tf.constant(0.0)
  #  w2v = np.array(embedding_vectors,dtype=np.float32)

    W1 = tf.Variable(w2v, trainable = False)
    X_embedding = tf.nn.embedding_lookup(W1, X)
    X_embedding = X_embedding[...,tf.newaxis] 


    poses, activations = capsule_model_B(X_embedding, args.num_classes)
    loss = margin_loss(y, activations) 
    y_pred = tf.argmax(activations, axis=1, name="y_proba")    
    correct = tf.equal(tf.argmax(y, axis=1), y_pred, name="correct")
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, name = 'opt'+str(fold_no))   
    training_op = optimizer.minimize(loss, name="training_op")
    gradients, variables = zip(*optimizer.compute_gradients(loss)) 
    with tf.Session() as sess:

      init = tf.global_variables_initializer()
      sess.run(init)

      for epoch in range(0,6): 

        for iteration in range(1, n_iterations_per_epoch + 1):
            
          
          X_batch, y_batch = mr_train1.next()          
          _, loss_train, probs, capsule_pose = sess.run(
                [training_op, loss, activations, poses],
                feed_dict={X: X_batch[:,:args.max_sent],
                          y: y_batch,
                          is_training: True,
                          learning_rate:lr,
                          margin:m})        
          print("\rIteration: {}/{} ({:.1f}%) epoch:{}  Loss: {:.5f}".format(iteration, n_iterations_per_epoch, iteration * 100 / n_iterations_per_epoch, epoch+1, loss_train), end="")                        
        preds_list, y_list = [], []
        for iteration in range(1, n_iterations_test + 1):
          X_batch, y_batch = mr_test1.next()             
          probs = sess.run([activations],
                    feed_dict={X:X_batch[:,:args.max_sent],
                                is_training: False})
          preds_list = preds_list + probs[0].tolist()
          y_list = y_list + y_batch.tolist()

        y_list = np.array(y_list)
        preds_probs = np.array(preds_list)  
        labels = np.argmax(y_list, axis=1)
        predictions = np.argmax(preds_probs, axis=1)

        accuracy_fold = accuracy_score(labels, predictions)
        precision_fold = precision_score(labels, predictions, average='weighted', zero_division = 0 )
        recall_fold = recall_score(labels, predictions, average='weighted')
        f1_fold = f1_score(labels, predictions, average='weighted')
        if best_f1 <= f1_fold :
          best_accuracy = accuracy_fold
          best_precision = precision_fold
          best_recall = recall_fold
          best_f1 = f1_fold
        

      acc_per_fold.append(best_accuracy)
      precision_per_fold.append(best_precision)
      recall_per_fold.append(best_recall)
      f1_per_fold.append(best_f1)
      print("\rFold: {} accuracy: {:.4f}%  Precision: {:.4f} recall: {:.4f} F1: {:.4f}".format(fold_no, best_accuracy, best_precision, best_recall, best_f1))
      if args.loss_type == 'margin_loss':    
            m = min(0.9, m + 0.1)
      fold_no += 1

accuracy = np.mean(acc_per_fold)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = np.mean(precision_per_fold)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = np.mean(recall_per_fold)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = np.mean(f1_per_fold)
print('F1 score: %f' % f1)

NameError: ignored